# Test B: Light‑curve microstructure form

Use this notebook to evaluate the presence of phase‑locked microstructure in two epochs of photometric monitoring.

## Inputs

Upload two comma‑separated value (CSV) files called `photometry_epoch1.csv` and `photometry_epoch2.csv` to the Colab runtime.  Each file must have at least the following columns:

- `time_mjd`: observation time in modified Julian day (UTC).
- Either `mag` (instrumental magnitude) or `flux` (relative flux).  If `mag` is provided the notebook converts it to relative flux by subtracting the median magnitude and exponentiating.
- `mag_err` or `flux_err`: uncertainty associated with each measurement.  If errors are not available provide a column of NaNs.

## Outputs

The notebook identifies the dominant period using a Lomb–Scargle periodogram, folds both epochs on this period, computes the semi‑amplitude of the folded light curves and reports whether the amplitude exceeds **0.5 %**.  It also plots the folded light curves for visual inspection.


In [ ]:
# Install required packages
import sys
!pip install --quiet numpy pandas matplotlib astropy > /dev/null

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle

# Helper to convert magnitudes to relative flux
def convert_to_flux(df):
    if 'mag' in df.columns:
        # convert mags to relative flux around median
        med_mag = np.nanmedian(df['mag'])
        flux = 10 ** (-0.4 * (df['mag'] - med_mag))
        df['flux'] = flux
        # propagate magnitude errors into flux error if available
        if 'mag_err' in df.columns:
            df['flux_err'] = df['flux'] * (np.log(10) * 0.4 * df['mag_err'])
        else:
            df['flux_err'] = np.nan
    elif 'flux' in df.columns:
        # ensure flux_err column exists
        if 'flux_err' not in df.columns:
            df['flux_err'] = np.nan
    else:
        raise ValueError('CSV must contain either a mag or flux column')
    return df

# Read the two epochs
df1 = pd.read_csv('photometry_epoch1.csv')
df2 = pd.read_csv('photometry_epoch2.csv')
df1 = convert_to_flux(df1)
df2 = convert_to_flux(df2)

# Normalise flux
df1['flux_norm'] = df1['flux'] / np.nanmedian(df1['flux'])
df2['flux_norm'] = df2['flux'] / np.nanmedian(df2['flux'])

# Concatenate data for period search
times = np.concatenate([df1['time_mjd'].values, df2['time_mjd'].values])
fluxes = np.concatenate([df1['flux_norm'].values, df2['flux_norm'].values])

# Set period search range in days (1–12 h = ~0.04–0.5 days)
min_period = 0.04  # days
max_period = 0.5   # days

# Lomb–Scargle periodogram
ls = LombScargle(times, fluxes)
frequency, power = ls.autopower(minimum_frequency=1/max_period, maximum_frequency=1/min_period)
best_freq = frequency[np.argmax(power)]
best_period = 1 / best_freq
print(f'Best period: {best_period * 24:.3f} hours')

# Fold each epoch
phase1 = (df1['time_mjd'] / best_period) % 1
phase2 = (df2['time_mjd'] / best_period) % 1

# Compute semi‑amplitude in percent
amp1 = (np.nanmax(df1['flux_norm']) - np.nanmin(df1['flux_norm'])) * 50
amp2 = (np.nanmax(df2['flux_norm']) - np.nanmin(df2['flux_norm'])) * 50
print(f'Semi‑amplitude epoch 1: {amp1:.3f}%')
print(f'Semi‑amplitude epoch 2: {amp2:.3f}%')

# Plot the folded light curves
plt.figure(figsize=(8,4))
plt.scatter(phase1, df1['flux_norm'], s=10, alpha=0.7, label='Epoch 1')
plt.scatter(phase2, df2['flux_norm'], s=10, alpha=0.7, label='Epoch 2')
plt.xlabel('Phase (cycles)')
plt.ylabel('Normalised flux')
plt.title('Folded light curves')
plt.legend()
plt.show()

# Evaluate microstructure criterion
if (amp1 > 0.5) and (amp2 > 0.5):
    print('Microstructure semi‑amplitude > 0.5% in both epochs. Candidate coherent microstructure.')
else:
    print('Amplitude below 0.5% threshold; microstructure not detected.')
